<h1><center>Effort Estimation</center></h1>

# Table of Contents

**[Business Understanding](#1)**
   - [Business Objective](#1.1)
   - [Current Solution & Alternatives](#1.2)
   - [Proposed Solution](#1.3)
   
**[Data Understanding & Cleaning](#2)**
  - [Import Libraries & Configure Notebook](#2.1)
  - [Import & Review Data](#2.2)
  - [Filter Out High Value Customer](#2.3)
  - [Label Churned Customers](#2.4)
  - [Delete Churned Phase data](#2.5)
  - [Columns With One Value Only](#2.6)
  - [Columns With Many Null values](#2.7)
  - [Columns With Mostly Zeroes](#2.8)
  - [Delete Unnecessary Columns](#2.9) 
  
**[Data Analysis & Preparation](#3)**
  - [Time Gap Between Recharges](#3.1)
  - [Attribute Relationships & Collinearity Analysis](#3.2) 
  - [Outliers](#3.3)
  - [Analyze Target Variable](#3.4)
  - [Bivariate Analysis](#3.5)
  - [Train-Test Split](#3.6)
  - [Rescale Variables](#3.7)
  - [Class Imbalance](#3.8)

  
**[Model Building & Evaluation](#4)**
   - [Logistic Regression Models](#4.1)
      - [Class Balancing & Baseline Model](#4.1.1)
      - [Optimum Probability Threshold](#4.1.1)
      - [Dimensionality Reduction with PCA](#4.1.2)
      - [Hyperparameter tuning](#4.1.3)
      - [Generalized Regression using Polynomial Features](#4.1.4)
      
   - [Decision Tree Models](#4.2)
      - [DT - Basic Model](#4.2.1)
      - [DT - Hyperparameter Tuning ](#4.2.2)
      - [Random Forests - Basic Model](#4.2.3)
      - [Random Forests - Hyperparameter Tuning](#4.2.4)
      
   - [Support Vector Machine Models](#4.3)
      - [Basic Model](#4.3.1)
      - [Hyperparameter Tuning](#4.3.2)
      
   - [XGBoost Models](#4.4)
      - [Basic Model](#4.4.1)
      - [Hyperparameter Tuning](#4.4.2)
      
   - [Model for Drivers Analysis](#4.5)
   - [Conclusion on Modelling](#4.6)

**[Business Presentation](#5)**
   - [Drivers of Churn](#5.1)
   - [Recommendations](#5.2)
 
**[References](#6)**



<a id="1"></a>
# Business Understanding

Raw Data Stats (prior to import)
- 409 archived projects - alltime
- 137 achived projects - 2019 to 13-july-2020

<a id="2"></a>
# Data Understanding & Cleaning

The dataset is from Harvest

<a id="2.1"></a>
## Import Libraries & Configure Notebook

In [ ]:
#Core libraries
import numpy as np
import pandas as pd

# Libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus, graphviz


#Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

#Modelling
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import Lasso
# from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
# from sklearn.feature_selection import RFE
# from statsmodels.stats.outliers_influence import variance_inflation_factor #For multicollinearity
from sklearn.decomposition import PCA, IncrementalPCA
# import xgboost as xgb
from xgboost import XGBClassifier
# from xgboost import plot_importance

#Model selection and evaluation tools
from sklearn.metrics import auc, roc_auc_score,roc_curve,f1_score,precision_recall_curve,classification_report, confusion_matrix,accuracy_score
# from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


C:\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
Using TensorFlow backend.


In [ ]:
#Configure notebook

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Set Parameters for Displaying data
pd.options.display.max_info_columns = 300
pd.set_option('display.max_columns', 120)
pd.options.display.max_rows = 300

np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.4f}'.format

# #InteractiveShell.ast_node_interactivity = "all"
sns.set_style("whitegrid")
%matplotlib inline


# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [3]:
data2020 = pd.read_excel('data/timesheets_2020.xlsx')
data2019 = pd.read_excel('data/timesheets_2019.xlsx')

In [4]:
data2020.shape
data2019.shape

(13831, 21)

(15891, 21)

In [5]:
data2020.head(2)
data2019.head(2)

,Date,Client,Project,Project Code,Task,Notes,Hours,Hours Rounded,Billable,Invoiced,Approved,First_Name,Last_Name,Roles,Employee,Billable Rate,Billable Amount,Cost Rate,Cost Amount,Currency,External_Reference_URL
0,2020-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,NaN,PTO / Holiday / Travel,NaN,8.0000,8.0000,No,No,No,Alexandra,Ciaccio,NA Pro Services|NA PS PMs,Yes,0.0000,0.0000,0,0,United States Dollar - USD,NaN
1,2020-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,NaN,PTO / Holiday / Travel,New Years Day,8.0000,8.0000,No,No,No,Sean,Keough,NA Pro Services,Yes,0.0000,0.0000,0,0,United States Dollar - USD,NaN


,Date,Client,Project,Project Code,Task,Notes,Hours,Hours Rounded,Billable,Invoiced,Approved,First_Name,Last_Name,Roles,Employee,Billable Rate,Billable Amount,Cost Rate,Cost Amount,Currency,External_Reference_URL
0,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,NaN,PTO / Holiday / Travel,NaN,8.0000,8.0000,No,No,No,Alexandra,Ciaccio,NA Pro Services|NA PS PMs,Yes,0.0000,0.0000,0,0,United States Dollar - USD,NaN
1,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,NaN,PTO / Holiday / Travel,NaN,8.0000,8.0000,No,No,No,Ian,Morrison,NaN,Yes,0.0000,0.0000,0,0,United States Dollar - USD,NaN


>**Observations**:
- Each row is a single timesheet entry.
- The 2020 data is up to 28-May and has 13k entries whereas all of 2019 has 15k entries presumably because there's greater timesheet discipline in 2019

Let's see whether timesheet discipline is only an International issue or NA as well.

In [6]:
data2019.Roles.unique()

array(['NA Pro Services|NA PS PMs', nan, 'NA Pro Services',
       'INTL Pro Services', 'SC Australia',
       'NA Pro Services|NA PS Subcontractors', 'INTL Customer Success',
       'NA Customer Success|NA CXMs', 'NA PS Subcontractors', 'NA CXMs'],
      dtype=object)

In [7]:
data2020.Roles.unique()

array(['NA Pro Services|NA PS PMs', 'NA Pro Services',
       'NA Pro Services|NA PS Subcontractors', 'INTL Pro Services',
       'NA Customer Success|NA CXMs', 'NA PS Subcontractors', nan,
       'INTL Customer Success', 'NA CXMs'], dtype=object)

In [8]:
#2020 billable hours
pd.pivot_table(data2020[data2020.Billable=='Yes'],values='Hours',aggfunc=np.sum,index='Roles').sort_values(by='Hours',ascending=False)

#ilink hours 
pd.pivot_table(data2020[(data2020.First_Name=='iLink 1')|(data2019.First_Name=='iLink 2')],values='Hours',aggfunc=np.sum,index='Roles').sort_values(by='Hours',ascending=False)

,Hours
Roles,
INTL Pro Services,4149.0800
NA Pro Services,2972.4700
NA Pro Services|NA PS PMs,1608.6100
NA Pro Services|NA PS Subcontractors,1336.2500
NA PS Subcontractors,170.5000
NA Customer Success|NA CXMs,31.0000
NA CXMs,13.1600
INTL Customer Success,5.5000


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,Hours
Roles,
NA Pro Services,67.7500
NA Pro Services|NA PS PMs,52.6700
INTL Pro Services,49.5000
NA PS Subcontractors,24.0000
NA Pro Services|NA PS Subcontractors,8.5000


In [9]:
#2019 billable hours
pd.pivot_table(data2019[(data2019.Billable=='Yes')],values='Hours',aggfunc=np.sum,index='Roles').sort_values(by='Hours',ascending=False)

#ilink hours
pd.pivot_table(data2019[(data2019.First_Name=='iLink 1')|(data2019.First_Name=='iLink 2')],values='Hours',aggfunc=np.sum,index='Roles').sort_values(by='Hours',ascending=False)

,Hours
Roles,
NA Pro Services,5529.7400
INTL Pro Services,3577.0900
NA Pro Services|NA PS PMs,2830.4200
NA PS Subcontractors,1504.0000
NA Pro Services|NA PS Subcontractors,1094.7500
INTL Customer Success,307.0800
NA Customer Success|NA CXMs,114.0000
SC Australia,75.2500
NA CXMs,10.5000


,Hours
Roles,
NA PS Subcontractors,1504.0000


>**Observations**: 
- We can safely ignore iLink hours in 2020.
- Intl region - In H1 2020 we have 15% more billings than all of 2019. Could be because of timesheet discpline.
- NA region - H1 2020 billable time is  42% of bookings of all of 2019 are done. Clearly, demand is weak in 2020.
- Intl is at 33% higher billable compared to NA in 2020

What are our top non-billable tasks in 2020?

In [10]:
pd.pivot_table(data2020[data2020.Billable=='No'],values='Hours',aggfunc=np.sum,index='Task').sort_values(by='Hours',ascending=False).head(10)

,Hours
Task,
"Non-Billable (receive training, meetings, updates i.e. roadmaps, new features, 1on1, team meetings, all-hands)",4124.3600
"Non-Billable (admin work, emails, time tracking, etc.)",3408.7200
PTO / Holiday / Travel,2654.9200
"Non-billable (writing recipes, best practices documentation)",928.6800
Admin,758.2500
Non-Billable (pre-sales & PS scoping),723.7500
Customer support activities,562.7500
External Training,449.0000
"Non-Billable (Assistance to LMI i.e. SCs, PS, Sales, CSMs, demo integrations)",390.0500


Let's concatenate the 2 tables now

In [11]:
data = pd.concat([data2019,data2020])

In [12]:
data.shape

(29722, 21)

In [13]:
#Drop columns that are not useful
data = data.drop(columns=['Project Code','External_Reference_URL','Currency','Cost Amount','Cost Rate','Billable Amount','Billable Rate','Employee','Hours Rounded','Approved','Invoiced','Last_Name'])

In [14]:
data.head(2)

,Date,Client,Project,Task,Notes,Hours,Billable,First_Name,Roles
0,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,PTO / Holiday / Travel,NaN,8.0000,No,Alexandra,NA Pro Services|NA PS PMs
1,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,PTO / Holiday / Travel,NaN,8.0000,No,Ian,NaN


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29722 entries, 0 to 13830
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        29722 non-null  datetime64[ns]
 1   Client      29722 non-null  object        
 2   Project     29722 non-null  object        
 3   Task        29722 non-null  object        
 4   Notes       10141 non-null  object        
 5   Hours       29722 non-null  float64       
 6   Billable    29722 non-null  object        
 7   First_Name  29722 non-null  object        
 8   Roles       29147 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 2.3+ MB


>**Observation**: A little over half the time logs are billable.

In [16]:
data.head()

,Date,Client,Project,Task,Notes,Hours,Billable,First_Name,Roles
0,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,PTO / Holiday / Travel,NaN,8.0000,No,Alexandra,NA Pro Services|NA PS PMs
1,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,PTO / Holiday / Travel,NaN,8.0000,No,Ian,NaN
2,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,PTO / Holiday / Travel,NaN,8.0000,No,Steve,NaN
3,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,PTO / Holiday / Travel,NaN,8.0000,No,Pablo,NA Pro Services
4,2019-01-01,2 - LogMeIn (NA),LogMeIn NA - Non-Billable Time,PTO / Holiday / Travel,NaN,8.0000,No,Sean,NA Pro Services


Top projects

In [17]:
pd.pivot_table(data2020[data2020.Billable=='Yes'],index=['Client','Project'],values='Hours',aggfunc=np.sum).sort_values(by='Hours',ascending=False).head(10)

,,Hours
Client,Project,
NA - BlueShield,BlueShield - Bot & Agent with SNOW Integration,823.7500
NA - Cox Communications,Cox Communications: Phase 2,663.2500
EMEA - Dyson,NOV19 - Dyson/Bold360ai/GlobalAIProject/650hrs/KAouidef,624.0800
2 - LogMeIn (NA),VIP Escalations/Bugs - North America,392.5000
EMEA - Vodafone UK,MAR20 Vodafone/Bold360ai/Customisation/337 hrs,283.2500
NA - Fannie Mae,Fannie Mae - 240 Hour Bucket ($60K) - Sept 2019 thru Aug 2020,240.5000
APAC - Fuji,Aug20 Fuji/Bold360ai/Support Centre/300hrs/BRajbhoy_MKesari_MShriki,232.0000
INDIA - Wipro,Dec20 Wipro_Olmypus/Boldai-agent-Rescue/320hrs/MKesari_JKartheek,227.7500
NA - Loews,Loews/Bold360ai/ServiceNow Bot Integration,187.5900


In [18]:
pd.pivot_table(data,index=['Client','Project'],values='Hours',aggfunc=np.sum).sort_values(by='Hours',ascending=False).head(10)

Hours
Client                  Project                                                      
2 - LogMeIn (NA)        LogMeIn NA - Non-Billable Time                     17923.8300
1 - LogMeIn (INTL)      LogMeIn INTL - Admin/Non-Billable Time              7368.1400
                        LMI - Nonbillable                                   1709.6700
                        DEC20 Allcustomers/Bold360ai/TimeOff                1275.5000
NA - CIBC               MAR19 CIBC / BoldChat / Email and live chat (20...  1222.2200
EMEA - Dyson            NOV19 - Dyson/Bold360ai/GlobalAIProject/650hrs/...  1049.8300
NA - CIBC               NA - CIBC - new SOW - web chat implementation (...  1003.6700
2 - LogMeIn (NA)        Holiday/Vacation                                     935.2500
NA - BlueShield         BlueShield - Bot & Agent with SNOW Integration       825.2500
NA - Cox Communications Cox Communications: Phase 2                          771.5000

Top individuals

In [24]:
pd.pivot_table(data2020[data2020.Billable=='Yes'],index=['First_Name'],values='Hours',aggfunc=np.sum).sort_values(by='Hours',ascending=False).head(10)

,Hours
First_Name,
Franco,686.5000
Luis,649.7500
Tim,550.0000
Pablo,504.5000
Alyssa,495.5000
Kevin,486.0000
Michele,452.0000
Mayank,445.2500
Kyle,441.7500


In [27]:
pd.pivot_table(data[data.Billable=='Yes'],index=['First_Name'],values='Hours',aggfunc=np.sum).sort_values(by='Hours',ascending=False).head(10)

,Hours
First_Name,
Tim,2210.8400
iLink 2,1674.5000
Sean,1392.6200
Alexandra,1364.4500
Luis,1228.5000
Pablo,1221.2500
Franco,1202.5000
Kevin,1201.7500
Alyssa,1172.5000


<a id="6"></a>
# References

- Estimation techniques - https://www.simplilearn.com/project-estimation-techniques-article#:~:text=While%20accurate%20estimates%20are%20the,Analysis%2C%20Reserve%20Analysis%2C%20Bottom%2D
- Parametrics estimation - https://dazeinfo.com/2019/10/08/mobile-data-usage-in-india-by-year-graphfarm/
- Top down vs bottom up approach - https://www.liquidplanner.com/blog/how-long-is-that-going-to-take-top-down-vs-bottom-up-strategies/
- Project estimation methods - https://www.liquidplanner.com/blog/5-methods-of-project-estimation/
- 3-point estimate & PERT (read the risk based explanation, it's fantastic) - https://www.projectengineer.net/three-point-estimating-for-the-pmp-exam/
- Parametric Estimating - https://www.dau.edu/tools/Lists/DAUTools/Attachments/264/Parametric%20Estimating_Linear%20Regression.pdf